# Investigation of old transplant data

To run this, please copy old patient data (`KDP` directory) and `LD_kidney_survival.csv` to `data` directory.

## Imports

In [ ]:
import pandas as pd
import os
from dataclasses import dataclass, asdict
from typing import List
import re
import numpy as np
import sys

In [ ]:
#filenames = sorted(os.listdir('KDP'))
#[(i + 10, 1, f) for i, f in enumerate(filenames)]

## A) Load old patients data

In [ ]:
@dataclass
class PatientPair:
    txm_event: str
    
    donor_name: str
    donor_blood_group: str
    donor_year_of_birth: int
    donor_relationship: str
    donor_typization: str
        
    recipient_name: str = ''
    recipient_blood_group: str = ''
    recipient_year_of_birth: str = ''
    recipient_typization: str = ''
        
    recipient_luminex_date: str = None
    recipient_luminex_1: str = None
    recipient_luminex_2: str = ''
    recipient_acceptable_blood: str = None

In [ ]:
@dataclass
class ExcelColumnsMap:
    donor_name: str
    donor_blood_group: str
    donor_year_of_birth: int
    donor_relationship: str
    donor_typization: str
        
    recipient_name: str
    recipient_blood_group: str
    recipient_year_of_birth: str
    recipient_typization: str
        
    recipient_luminex_date: str
    recipient_luminex_1: str
    recipient_luminex_2: str
    recipient_acceptable_blood: str
    
map_0 = None

map_1 = ExcelColumnsMap(
    ['Donor', 'DÁRCE'],
    ['BLOOD GROUP donor', 'KS', 'KS dárce','KS DÁRCE', 'Blood group', 'KS DÁRCE donor', 'KS  donor'],
    ['ID/age', 'ID', 'RČ'],
    ['RELATIONSHIP', 'VZTAH'],
    ['TYPIZATION DONOR', 'TYPIZACE', 'Typizace dárce', 'Typing', 'TYPIZACE DÁRCE'],
    
    [' RECIPIENT', 'RECIPIENT', 'PŘÍJEMCE'],
    ['BLOOD GROUP recipient', 'KS.1', 'KS příjemce','KS PŘÍJEMCE', 'Blood group.1'],
    ['ID/age.1', 'ID.1', 'RČ.1'], 
    ['TYPIZATION RECIPIENT', 'TYPIZACE.1', 'Typizace příjemce', 'Typing.1', 'TYPIZACE PŘÍJEMCE'],
    
    ['datum luminex'],
    ['luminex varianta 1', 'Luminex varianta 1', 'LUMINEX: SA1>1000MFI,SA2>2000MFI', 'Luminex'],
    ['luminex  cut-off (2000 MFI) varianta 2', 'LUMINEX:SA1 a SA2>2000MFI', 'Luminex varianta 2', 'luminex posun cut-off', 'luminex posun cut off', 'luminex posun cut-off varianta 2'],
    ['Acceptable blood group', 'Povolené KS', 'allowed blood group', 'povolené KS.'],
)

In [ ]:
import math

def _parse_hla(hla_codes_str: str):
    if 'neg' in hla_codes_str.lower():
        return "" #[]
    # remove codes in brackets, they are only in detail all the split codes for broade in front of the bracket
    hla_codes_str = re.sub(r'\(.*?\)', '', hla_codes_str)
    hla_codes = re.split('[,. ()]+', hla_codes_str)
    hla_codes = [code.upper() for code in hla_codes if len(code) > 0]

    return " ".join(sorted(hla_codes))

def row_to_patient_pair(row, config):
    txm_event, header, sheet_name, nrows, col_map, filename = config
    
    col_map_dict = dict()
    for key, col_names in asdict(col_map).items():
        col_map_dict[key] = None
        for col_name in col_names:
            col_name = col_name.lower()
            #print(set(row.keys()))
            if col_name in set(row.keys()):
                col_map_dict[key] = str(row[col_name]).strip().replace('    ', ' ')
                continue
    
    missing_columns = {k: asdict(col_map)[k] for k, v in col_map_dict.items() 
                       if v is None and k in { # required columns
                           'donor_blood_group', 'donor_typization', 'recipient_blood_group', 'recipient_typization', 'recipient_luminex_2', 'recipient_acceptable_blood'
                       }}
    if len(missing_columns) > 0:
        print(f'Missing columns in {filename}: {missing_columns}.\n -- {set(row.keys())}')
        #raise ValueError(f'Missing columns: {missing_columns}. {filename}: {set(row.keys())}')

    if pd.isnull(col_map_dict['donor_typization']) or pd.isnull(col_map_dict['recipient_typization']):
        # This happens for and bridging or if row is empty. Uncomment to check it
        # print(f"Warning: Row ignored ({filename}): {row}")
        return None
    
    col_map_dict['donor_typization'] = _parse_hla(col_map_dict['donor_typization'])
    col_map_dict['recipient_typization'] = _parse_hla(col_map_dict['recipient_typization'])
    if col_map_dict['recipient_luminex_1'] is not None:
        col_map_dict['recipient_luminex_1'] = _parse_hla(col_map_dict['recipient_luminex_1'])
    col_map_dict['recipient_luminex_2'] = _parse_hla(col_map_dict['recipient_luminex_2'])
    
    return PatientPair(
        txm_event,
        
        **col_map_dict
    )

In [ ]:
configs = [
    (10, 1, 0, None, map_1, 'PV10_prehled.xlsx'),
    (11, 1, 0, None, map_1, 'PV11_prehled_vysetreni.xlsx'),
    (12, 1, 0, None, map_1, 'PV12finální přehled.xlsx'),
    (13, 1, 2, None, map_1, 'PV13finalni přehled vyšetření.xlsx'),
    (14, 0, 0, 5, map_1, 'PV14_Czech_Vienna_final_results.xlsx'),
    (15, 1, 0, None, map_1, 'PV15.xlsx'),
    (16, 1, 0, None, map_1, 'PV16.xlsx'),
    (17, 1, 0, None, map_1, 'PV17.xlsx'),
    (18, 2, 0, None, map_1, 'PV18.xlsx'),
    (19, 0, 0, 8, map_1, 'PV19.xlsx'),
    (20, 0, 0, 9, map_1, 'PV20 + Vienna.xlsx'),
    (21, 0, 0, 11, map_1, 'PV21.xlsx'),
    (22, 0, 0, None, map_1, 'PV22.xlsx'),
    (23, 0, 0, 13, map_1, 'PV23.xlsx'),
    (24, 0, 0, 12, map_1, 'PV24.xlsx'),
    (25, 0, 0, 17, map_1, 'PV25.xlsx'),
    # (26, 1, 'PV26 + Izrael.xlsx'),
    (26, 1, 0, 15, map_1, 'PV26.xlsx'),
    (27, 1, 0, 11, map_1, 'PV27.xlsx'),
    #(29, 1, 'PV28 + Izrael.xlsx'),
    (28, 1, 0, 15, map_1, 'PV28 + Rakousko.xlsx'),
    (29, 1, 0, 13, map_1, 'PV29.xlsx'),
    (30, 1, 0, 13, map_1, 'PV30.xlsx'),
    #(33, 1, 'Přehled vyšetření k 14.1.2015.xlsx'),
    #(34, 1, 'Souhrn vysetreni k 23.5.2013.xls'),
    #(35, 1, 'Stav vyšetření k 10.6.2014.xlsx'),
    #(36, 1, 'Stav vyšetření k 13.02.2014.xls'),
    #(37, 1, 'Stav vyšetření k 20.11.2013.xls'),
    #(38, 1, 'Stav vyšetření k 4.9.2013.xls'),
    #(39, 1, 'Stav vyšetření k 4.9.2014.xlsx')
]

In [ ]:
patient_pairs = []

for config in configs[:]:
    txm_event, header, sheet_name, nrows, col_map, filename = config
    
    path = os.path.join('data/KDP', filename)
    df = pd.read_excel(path, index_col=None, header=header, sheet_name=sheet_name, nrows=nrows)
    # print(df.columns)
    if txm_event == 14:
        df = df.rename(columns={
            'Typing ': 'typing',
            'Typing': 'typing.1',
            'LUMINEX results': 'luminex varianta 2'  # This needs to be consulted with imunologists
        })
    if txm_event == 10:
        df['acceptable blood group'] = ''
        
    df = df.rename(columns=lambda x: x.lower())
    
    for _, row in df.iterrows():
        if col_map is None:
            continue
        pp = row_to_patient_pair(row, config)
        if pp is not None:
            patient_pairs.append(pp)

patient_pairs = [asdict(pp) for pp in patient_pairs]

df_all_patients = pd.DataFrame(patient_pairs)

# All patients for all txm events
df_all_patients.head()
#df_all_patients.loc[df_all_patients.txm_event == 28]

# I checked in xlsx files that the data were parsed to the last czech patient
# df_all_patients.groupby(['txm_event']).last()

In [ ]:
print(len(df_all_patients.index))
print("---")

print(len(df_all_patients.groupby(['donor_name']).first().index))
print(len(df_all_patients.groupby(['donor_typization']).first().index))
print(len(df_all_patients.groupby(['donor_name', 'donor_typization']).first().index))
print("---")

print(len(df_all_patients.groupby(['recipient_name']).first().index))
print(len(df_all_patients.groupby(['recipient_typization']).first().index))
print(len(df_all_patients.groupby(['recipient_name', 'recipient_typization']).first().index))

print("---")
print(len(df_all_patients.groupby(['recipient_luminex_2']).first().index))
print(len(df_all_patients.groupby(['recipient_luminex_2', 'recipient_typization']).first().index))

### Export unique patients

In [ ]:
# pd.set_option('display.max_rows', 500)
# df_unique_patients = df_all_patients[~df_all_patients.recipient_year_of_birth.isin(['nan', 'x'])].groupby(['recipient_name', 'recipient_year_of_birth']).first()[[]]
# df_unique_patients.to_csv('patients_list.csv')
# display(df_unique_patients)
# pd.reset_option('display.max_rows')

### Problematic data

#### 1. Donors with the same name but different typization

In [ ]:
_problematic_patients = df_all_patients.set_index('donor_name').join(df_all_patients.set_index('donor_name'), lsuffix='_l', rsuffix='_r')\
    .loc[lambda x: x.donor_typization_l < x.donor_typization_r]

print(len(_problematic_patients.index))
_problematic_patients[['txm_event_l', 'donor_typization_l', 'donor_typization_r', 'txm_event_r', ]].head()

#### 2. Recipients with the same name but different typization

In [ ]:
_problematic_patients = df_all_patients.set_index('recipient_name').join(df_all_patients.set_index('recipient_name'), lsuffix='_l', rsuffix='_r')\
    .loc[lambda x: x.recipient_typization_l < x.recipient_typization_r]

print(len(_problematic_patients.index))
_problematic_patients[['txm_event_l', 'recipient_typization_l', 'recipient_typization_r', 'txm_event_r', ]].head()

#### 3. Donors with the same typization but different name

In [ ]:
_problematic_patients = df_all_patients.set_index('donor_typization').join(df_all_patients.set_index('donor_typization'), lsuffix='_l', rsuffix='_r')\
    .loc[lambda x: x.donor_name_l < x.donor_name_r]

print(len(_problematic_patients.index))
_problematic_patients[['txm_event_l', 'donor_name_l', 'donor_name_r', 'txm_event_r', ]].head()

#### 4. Recipients with the same typization but different name

In [ ]:
_problematic_patients = df_all_patients.set_index('recipient_typization').join(df_all_patients.set_index('recipient_typization'), lsuffix='_l', rsuffix='_r')\
    .loc[lambda x: x.recipient_name_l < x.recipient_name_r]

print(len(_problematic_patients.index))
_problematic_patients[['txm_event_l', 'recipient_name_l', 'recipient_name_r', 'txm_event_r', ]].head()

### Plotting

In [ ]:
%matplotlib inline

a = df_all_patients.groupby(['recipient_name']).count().groupby(['txm_event']).count()['donor_blood_group']
a.plot.bar(title='# patients in # rounds', figsize=(10, 5))

In [ ]:
df_all_patients.groupby(['txm_event']).nunique()[['recipient_typization', 'recipient_name','donor_typization', 'donor_name' ]]\
    .plot.bar(figsize=(20, 5), title='Number of specified values for each txm_event')

In [ ]:
df_patients_to_event = pd.pivot_table(df_all_patients.assign(one=1), values='one', index=['recipient_name'], columns=['txm_event'], aggfunc=np.sum, fill_value=0)
df_event_to_patients = pd.pivot_table(df_all_patients.assign(one=1), values='one', index=['txm_event'], columns=['recipient_name'], aggfunc=np.sum, fill_value=0)
df_event_to_patients.plot.area(figsize=(20,10), legend=False, title='In what txm events each patient was')

In [ ]:
# Uncomment to show the above in table
# df_patients_to_event

## B) Kidney survival data

In [ ]:
df_survival = pd.read_csv('data/LD_kidney_survival.csv')
df_survival.head()

In [ ]:
# df_survival.columns

In [ ]:
# df_survival.loc[0]

In [ ]:
def _create_hlas(group_to_values):
    all_codes = ",".join([
        ",".join(
            [
                group + str(int(val)) # + ('FOO' if val != str(int(val)) else '') # Problem: types with leading zeror are not joined even though if the zero is removed
                for val in str(values if not pd.isnull(values) else '').split(",") 
                if len(val.strip()) > 0 and val != '-'
            ]
        )
        for group, values in group_to_values.items()
    ])
    return _parse_hla(all_codes)

def create_donor_typization(row):
    return _create_hlas({
        'A': row['Donor_HLAA'],
        'B': row['Donor_HLAB'],
        'Cw': row['Donor_HLACw'],
        'DP': row['Donor_HLADP'],
        'DQ': row['Donor_HLADQ'],
        'DR': row['Donor_HLADR']
    })

def create_recipient_typization(row):
    return _create_hlas({
        'A': row['HLAA'],
        'B': row['HLAB'],
        'Cw': row['HLACw'],
        'DP': row['HLADP'],
        'DQ': row['HLADQ'],
        'DR': row['HLADR']
    })

df_survival['donor_typization'] = df_survival.apply(create_donor_typization, axis=1)
df_survival['recipient_typization'] = df_survival.apply(create_recipient_typization, axis=1)

df_survival['StartDate'] = pd.to_datetime(df_survival['StartDate'])
df_survival['LastVisitDate'] = pd.to_datetime(df_survival['LastVisitDate'])
df_survival['EndDate'] = pd.to_datetime(df_survival['EndDate'])

df_survival['delay'] = (df_survival['LastVisitDate'] - df_survival['StartDate']).dt.days

df_survival.head()
# df_survival.columns

### Not successful transplants with smallest delay

In [ ]:
df_survival.loc[df_survival.EndReason.notnull()].sort_values(by='delay').head()

## C) Join both data

### C.1. Preprocess patiens

In [ ]:
# df_all_patients

In [ ]:
df_patients_summary = df_all_patients.groupby(
    ['donor_name', 'donor_typization', 'donor_blood_group', 'donor_relationship', 'recipient_name', 
     'recipient_typization', 'recipient_acceptable_blood']
)[['txm_event']].agg(lambda x: ",".join([str(i) for i in x])).reset_index()

df_patients_summary.rename(columns={
    'donor_name': 'orig_donor_name',
    'donor_typization': 'orig_donor_typization',
    'donor_blood_group': 'orig_donor_blood_group',
    'donor_relationship': 'orig_donor_relationship',
    # 'recipient_luminex_2': 'recipient_luminex',
    'txm_event': 'txm_events'
}, inplace=True)

df_patients_summary['last_txm_event'] = df_patients_summary['txm_events'].apply(lambda events: int(str(events).split(",")[-1])).astype('int32')

print(len(df_patients_summary.index))
df_patients_summary.head()

### C.2. Preprocess survival data

In [ ]:
df_survival_summary = df_survival[['StartDate', 'LastVisitDate', 'EndDate', 'delay', 'EndReason', 'NoVisits', 'donor_typization', 'recipient_typization']]
df_survival_summary.head()

### C.3. Join both data

In [ ]:
df_joined = df_patients_summary.set_index('recipient_typization').join(df_survival_summary.set_index('recipient_typization'), how='inner')
df_joined = df_joined.reset_index()
print(len(df_joined.index))


### Oldest transplant in joined data

In [ ]:
df_joined_oldest = df_joined.sort_values(by='StartDate')
oldest_start_date = df_joined_oldest.iloc[0].StartDate
print(oldest_start_date)
df_joined_oldest.iloc[0:1]

### Number survival data that are more recent than the oldes transplant

In [ ]:
df_survival_summary_recent = df_survival_summary.loc[df_survival_summary.StartDate >= oldest_start_date].sort_values(by='StartDate')
print(len(df_survival_summary_recent.index))

In [ ]:
df_joined #.sort_values(by='StartDate')

In [ ]:
df_joined.plot(x='last_txm_event', y='StartDate', style=".", figsize=(10, 7))

### Duplicated data

In [ ]:
df_joined_duplicated = df_joined.reset_index()[df_joined.reset_index().duplicated(subset='recipient_typization')]
df_joined_duplicated

In [ ]:
df_joined_duplicated.plot(x='last_txm_event', y='StartDate', style=".", figsize=(10, 7))

## D) Run scorer

### Imports

In [ ]:
sys.path.insert(0, "../..")

from txmatching.scorers.split_hla_additive_scorer import SplitScorer
from txmatching.scorers.high_res_hla_additive_scorer import HighResScorer
from txmatching.scorers.high_res_other_hla_types_additive_scorer import \
    HighResWithDQDPScorer

from txmatching.utils.blood_groups import BloodGroup
from txmatching.utils.country_enum import Country

from txmatching.patients.patient import Donor, Recipient
from txmatching.patients.patient_parameters import PatientParameters

from tests.test_utilities.hla_preparation_utils import (create_antibodies,
                                                        create_hla_typing)

from tests.test_utilities.prepare_app_for_tests import DbTests

In [ ]:
# Initialize db
test = DbTests()
test.setUp()

In [ ]:
# test.tearDown()

In [ ]:
# Select scorer that will be used

split_scorer = SplitScorer()
high_res_scorer = HighResScorer()
high_res_other_hla_types_scorer = HighResWithDQDPScorer()

active_scorer = split_scorer

In [ ]:
def row_to_score(row):
    
    if row.donor_typization == '' or row.recipient_typization == '':
        return None
    
    donor_typing = row.donor_typization.split(" ")
    recipient_typing = row.recipient_typization.split(" ")
    
    donor = Donor(
        db_id=-1,
        medical_id='donor',
        related_recipient_db_id=1,
        parameters=PatientParameters(
            blood_group=BloodGroup.A,
            country_code=Country.CZE,
            hla_typing=create_hla_typing(
                donor_typing
            )
        )
    )
    
    recipient = Recipient(
        db_id=1,
        acceptable_blood_groups=[],
        related_donor_db_id=1,
        medical_id='recipient',
        parameters=PatientParameters(
            blood_group=BloodGroup.A,
            country_code=Country.CZE,
            hla_typing=create_hla_typing(recipient_typing)
        ),
        hla_antibodies=create_antibodies([])
    )
    
    score = active_scorer.score_transplant(
        donor=donor,
        recipient=recipient,
        original_donor=None
    )
    
    return score

df_scored = df_survival_summary
df_scored['score'] = df_scored.apply(row_to_score, axis=1)

### Score distribution

In [ ]:
df_scored_filtered = df_scored[df_scored.score.notnull()]
df_scored_filtered.score.hist(bins=26)

### Delay distribution on ended patients

In [ ]:
df_scored_ended = df_scored_filtered[df_scored_filtered.EndDate.notnull()]
print(len(df_scored_ended.index))
df_scored_ended.delay.hist()

### Score distribution on ended patients

In [ ]:
df_scored_ended.score.hist(bins=26)

### Score distribution on ended patients with delay < 10 years

In [ ]:
df_scored_ended[df_scored_ended.delay<3650].score.hist(bins=26)

### Dependency between delay and score on ended patients

In [ ]:
df_scored_ended.plot.scatter(x='delay', y='score')

In [ ]:
df_scored_ended.score.corr(df_scored_ended.delay)

### Ended patients with the smalles delay

In [ ]:
df_scored_ended.sort_values(by='delay', ascending=True).head()